<a href="https://colab.research.google.com/github/Jorge2018/DataScience2022-2/blob/main/Script_boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

url = "https://github.com/Jorge2018/RepositorioArchivos/blob/main/census_income.xlsx?raw=true"

response = requests.get(url)

dest = 'local-file.xlsx'

with open(dest, 'wb') as file:
    file.write(response.content)

df = pd.read_excel(dest)

df.head()


,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income-class
0,0.0,39.0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,1.0,50.0,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,2.0,38.0,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,3.0,53.0,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,4.0,28.0,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


**Prepara datos para modelar**

In [48]:
# Drop unnecessary column
df.drop(columns = 'Unnamed: 0', inplace = True)
# Binarize target column
df['income-class'] = df['income-class'].map({'<=50K': 1, '>50K': 0})
df['sex'] = df['sex'].map({'Male': 1, 'Female': 0})

**Exploracion de datos**

In [49]:
df.duplicated()
df = df.drop_duplicates()
print('la cantidad de registros sin duplicados es:\n',df.count())
print('---------------------------')
print('los datos faltantes del df son:\n',df.isna().sum())
print('---------------------------')
print('los tipos de datos son:\n')
df.info()
print('---------------------------')
df.describe().round(3)

la cantidad de registros sin duplicados es:
 age               29096
workclass         29096
education         29096
marital-status    29096
occupation        29096
relationship      29096
race              29096
sex               29096
capital-gain      29096
capital-loss      29096
hours-per-week    29096
native-country    29096
income-class      29096
dtype: int64
---------------------------
los datos faltantes del df son:
 age               0
workclass         0
education         0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income-class      0
dtype: int64
---------------------------
los tipos de datos son:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29096 entries, 0 to 32560
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             29096 non-null  float64

,age,sex,capital-gain,capital-loss,hours-per-week,income-class
count,29096.000,29096.000,29096.000,29096.000,29096.000,29096.000
mean,39.251,0.661,1197.802,97.175,40.638,0.752
std,13.687,0.473,7778.225,424.008,12.735,0.432
min,17.000,0.000,0.000,0.000,1.000,0.000
25%,28.000,0.000,0.000,0.000,40.000,1.000
50%,38.000,1.000,0.000,0.000,40.000,1.000
75%,48.000,1.000,0.000,0.000,45.000,1.000
max,90.000,1.000,99999.000,4356.000,99.000,1.000


**Separa entre X, y**

In [50]:
# Train test split
# Save X data
X = df.drop(columns = 'income-class')
# Encode our target
y = df['income-class']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

**Porceso para realizar transformaciones en los datos**

In [51]:
## **Instanciar selectores**
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

## Instanciar transformadores SimpleImputer, StandardScaler y OneHotEncoder
# Imputers
mean_imputer = SimpleImputer(strategy='mean')
freq_imputer = SimpleImputer(strategy='most_frequent')
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

## **Instanciar el pipeline**
numeric_pipe = make_pipeline(mean_imputer, scaler)
categorical_pipe = make_pipeline(freq_imputer, ohe)

## **Asignacion de tupla e instanciacion de ColumnTransformer**
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple)

## **Transformacion de datos**
# fit on train
preprocessor.fit(X_train,X_test)
# transform train and test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

**ajusta y entrena el modelo (Gradient, LGBM, xgb)**

In [52]:
# Instantiate and fit gradient boosting classifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train_processed, y_train)

# Instantiate and fit gradient boosting classifier
lgbm = LGBMClassifier()
lgbm.fit(X_train_processed, y_train)

# Instantiate and fit gradient boosting classifier
xgb = XGBClassifier()
xgb.fit(X_train_processed, y_train)





XGBClassifier()

**Evaluacion del modelo**

In [53]:
print('Training accuracy (Gradient):', gbc.score(X_train_processed, y_train))
print('Testing accuracy  (Gradient):', gbc.score(X_test_processed, y_test))

print('Training accuracy (LGBM):', lgbm.score(X_train_processed, y_train))
print('Testing accuracy  (LGBM):', lgbm.score(X_test_processed, y_test))

print('Training accuracy (xgb):', xgb.score(X_train_processed, y_train))
print('Testing accuracy  (xgb):', xgb.score(X_test_processed, y_test))




Training accuracy (Gradient): 0.8642654202181285
Testing accuracy  (Gradient): 0.8640362936486115
Training accuracy (LGBM): 0.8828246723490056
Testing accuracy  (LGBM): 0.8735221336266153
Training accuracy (xgb): 0.8605077444780497
Testing accuracy  (xgb): 0.8611492988726973
